# ER131 First Draft Project Template

Fall 2022

## How to use this notebook: 
* Follow the format and sequencing of sections
* Edit the markdown cells to be specific to your project.
* Put the code you use to accomplish the tasks of each section in between each markdown cell.
* Delete this cell before you hand in the notebook
* As with homeworks, you'll hand in a `.ipynb` file and a `.pdf` file; we'll also ask you to make your data available.

## Basic First Draft Requirements 
**Grading**.  You'll see point allocations listed in each of the section titles below.  In addition, there are other categories for points: 
1. (4 points) Build your first dataframe including at least on target variable and a set of features.  In a markdown cell, clearly explain what the target variable is, what each row represents, and what the features are.  Provide a summary of the entries in the dataframe (e.g. using a few plots and `pd.describe`)
2. (4 points) Choose and justify a type of prediction model to train.  (E.g. lasso, or random forests.)
4. (4 points) Generate a test-train split.  Then cross validate the model with the training data to tune at least one hyperparameter.
3. (4 points) Evaluate how well the prediction model performs on test data. 
5. (4 points) For clarity.  Use markdown cells to clearly and concisely explain what you are doing and your justification for your decisions.  Keep your code clean and readable and well commented.  

We are looking for progress and not perfection. In each category we'll score 1 point for completion, 2 points for demonstrating some understanding, 3 points for just a few mistakes, 4 points for demonstrating mastery.

## (delete this markdown cell in your final submission)

# YOUR TITLE HERE

In this cell, replace the text above with your project title.  

Then give an alphabetical (by last name) list of student group members.  Beside each student's name, provide an estimate of the number of hours each student put into the effort, anbd a description of each student's contribution to the project.  For example:

* Duncan Callaway (2 hours). developed template including overall workflow, specific tasks, and scoring rubric
* Julia Longmate (1 hours). checked template for clarity and provided suggestions on improvements

If one person did most of the work, or one person did very little work, talk openly about that as a team and make a plan to even the workload going forward.  Contact Duncan or Julia if you are not able to agree on how much work each person did, or to make a plan going forward.

## Build your first dataframe (4 points)
Build your first dataframe including at least on target variable and a set of features.  In a markdown cell, clearly explain what the target variable is, what each row represents, and what the features are.  Provide a summary of the entries in the dataframe (e.g. using a few plots and `pd.describe`).

This section should briefly explain where you got your data and what you've done to manipulate it.  The final project will ask for much more information here.

## Prediction model (4 points)
Choose and justify a prediction model for this application.  This section can be short, but well-reasoned!

## Training and cross validation (4 points)
Generate a test-train split.  Then train and cross validate your model with the training data to tune at least one hyperparameter.

## Evaluation and next steps (4 points)
Evaluate how well the prediction model performs on test data.  Rationalize the metric(s) you use to measure performance.  Be critical of your work!  It's ok if the model performs poorly. Talk about next steps.   

## Clarity (4 points)
This cell is here to make sure you don't forget.  You can delete it before you hand in your assignment.  

Use markdown cells to clearly and concisely explain what you are doing and your justification for your decisions.  Keep your code clean and readable and well commented.  